In [8]:
!cd C:\Users\TIF-AWaDH\Downloads\Transformer

In [9]:
!python -m venv mistral-gpu-env

In [11]:
!.\mistral-gpu-env\Scripts\activate

In [1]:
!pip uninstall -y torch torchvision torchaudio

In [2]:
!python --version
 


Python 3.9.0


In [1]:
!python -m ipykernel install --user --name=mistral-310 --display-name "Python (mistral-310)"


Installed kernelspec mistral-310 in C:\Users\TIF-AWaDH\AppData\Roaming\jupyter\kernels\mistral-310


In [2]:
!pip install --prefer-binary torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
!python -m pip install --upgrade pip setuptools wheel


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 524.3 kB/s eta 0:00:02
   ----------------- ---------------------- 0.5/1.2 MB 524.3 kB/s eta 0:00:02
   -------------------------- ------------- 0.8/1.2 MB 559.5 kB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 599.4 kB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 599.4 kB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 599

In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [1]:
!cd C:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\Scripts\activate.ps1

The directory name is invalid.


In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")



Torch version: 2.1.2+cu118
CUDA available: True
CUDA version: 11.8
Device name: NVIDIA GeForce RTX 3090


In [1]:
!pip install -r requirements.txt



You should consider upgrading via the 'C:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import transformers, trl, peft, accelerate, tokenizers, torch
print(transformers.__version__)
print(trl.__version__)
print(peft.__version__)
print(accelerate.__version__)
print(tokenizers.__version__)
print(torch.__version__, torch.cuda.is_available())



c:\Users\TIF-AWaDH\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.
c:\Users\TIF-AWaDH\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\TIF-AWaDH\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvis

4.53.2
0.20.0.dev0
0.7.0
1.8.1
0.21.2
2.7.1+cpu False


In [1]:
!pip install transformers accelerate datasets peft trl bitsandbytes xformers jupyter

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install peft==0.5.0 accelerate==0.21.0





  Attempting uninstall: accelerate

    Found existing installation: accelerate 0.27.2

    Uninstalling accelerate-0.27.2:

      Successfully uninstalled accelerate-0.27.2

   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
   ---------------------------------------- 0/2 [accelerate]
  Attempting uninstall: peft
   ---------------------------------------- 0/2 [accelerate]
    Found existing installation: peft 0.8.2
   ---------------------------------------- 0/2 [accelerate]
    Uninstalling peft-0.8.2:
   ---------------------------------------- 0/2 [accelerate]
      Successfully uninstalled peft-0.8.2
   ---------------------------------------- 0/2 [accelerate]
   -------------------- ------------------- 1/2 [peft]
   ---------------------------------------- 2/2 [peft]



In [ ]:
from huggingface_hub import login

login("your-token-here") 

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from datasets import load_dataset

# 1️⃣  Model + tokenizer (needs Hugging‑Face gated access)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,      # QLoRA
    device_map="auto",
    trust_remote_code=True
)

# 2️⃣  LoRA config
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_cfg)

# 3️⃣  Tiny sample dataset (replace with your own)
dataset = load_dataset("Abirate/english_quotes", split="train[:200]")
dataset = dataset.map(lambda e: {
    "text": f"### Instruction:\nGenerate a quote.\n\n### Response:\n{e['quote']}"
})

# 4️⃣  TrainingArguments
args = TrainingArguments(
    output_dir="./mistral-lora-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,                # RTX 3090 supports FP16
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
)

# 5️⃣  Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    args=args,
)

trainer.train()

# 6️⃣  Save LoRA adapters
trainer.model.save_pretrained("mistral-lora-checkpoint")
tokenizer.save_pretrained("mistral-lora-checkpoint")


c:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Exception: data did not match any variant of untagged enum PyPreTokenizerTypeWrapper at line 40 column 3

In [ ]:
from transformers.utils import cached_file
from huggingface_hub import snapshot_download
snapshot_download("mistralai/Mistral-7B-Instruct-v0.1", local_dir="./hf_cache")

# Force re-download
# snapshot_download("mistralai/Mistral-7B-Instruct-v0.1", local_dir="./hf_cache", force_download=True)


c:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 15 files:  33%|███▎      | 5/15 [00:27<00:55,  5.54s/it]

In [4]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '6874ed445060681881c737ad', 'name': 'Vanshika1108', 'fullname': 'Vanshika Sharma', 'email': 'vansharidhi1975@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/os0kz25n3Bw5N77fPYI8w.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'mistral-7B', 'role': 'read', 'createdAt': '2025-07-14T11:50:21.940Z'}}}


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes", split="train[:200]")

dataset = dataset.map(lambda e: {
    "text": f"### Instruction:\nGenerate a quote.\n\n### Response:\n{e['quote']}"
}).remove_columns(["quote", "author", "tags"])


In [4]:
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes", split="train[:200]")

dataset = dataset.map(lambda e: {
    "text": f"### Instruction:\nGenerate a quote.\n\n### Response:\n{e['quote']}"
}).remove_columns(["quote", "author", "tags"])


In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from trl import SFTTrainer

# 1️⃣ Load tokenizer and base model (no quantization)
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(base_model_name)

# 2️⃣ LoRA Configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

# 3️⃣ Load Dataset
dataset = load_dataset("Abirate/english_quotes", split="train[:200]")  # or your own

# 4️⃣ Format & Tokenize
def format(example):
    return {"text": example["quote"]}

dataset = dataset.map(format)

# Pack sequences for efficient training
training_args = TrainingArguments(
    output_dir="./tinyllama-lora-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,  # ✅ Important since you're using RTX 3090
    report_to="none"
)

# 5️⃣ Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    packing=True,  # ✅ to auto-handle input formatting and padding
    dataset_text_field="text"
)

trainer.train()

# 6️⃣ Save LoRA adapter and tokenizer
trainer.model.save_pretrained("tinyllama-lora-checkpoint")
tokenizer.save_pretrained("tinyllama-lora-checkpoint")




Generating train split: 7 examples [00:00, 625.66 examples/s]
  0%|          | 0/50 [02:23<?, ?it/s]
c:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
                                      
100%|██████████| 3/3 [00:06<00:00,  2.31s/it]


{'train_runtime': 6.9219, 'train_samples_per_second': 3.034, 'train_steps_per_second': 0.433, 'train_loss': 1.772229830423991, 'epoch': 3.0}


c:\Users\TIF-AWaDH\Downloads\Transformer\mistral-310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('tinyllama-lora-checkpoint\\tokenizer_config.json',
 'tinyllama-lora-checkpoint\\special_tokens_map.json',
 'tinyllama-lora-checkpoint\\tokenizer.json')

In [15]:
trainer.model.save_pretrained("tinyllama-lora-checkpoint")
tokenizer.save_pretrained("tinyllama-lora-checkpoint")


('tinyllama-lora-checkpoint\\tokenizer_config.json',
 'tinyllama-lora-checkpoint\\special_tokens_map.json',
 'tinyllama-lora-checkpoint\\tokenizer.json')

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("tinyllama-lora-checkpoint")
tokenizer = AutoTokenizer.from_pretrained("tinyllama-lora-checkpoint")


In [18]:
inputs = tokenizer("What do you do?", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


What do you do?

JASON: (smiling) I'm a writer.

SARAH: (smiling back) I'm a photographer.

JASON: (laughing) You're both very


In [13]:
!pip uninstall bitsandbytes


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # uses GPU
    torch_dtype=torch.float16  # instead of 4-bit quantization
)

inputs = tokenizer("Hello! Can you help me?", return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=30)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Hello! Can you help me?


In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("Torch device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


CUDA available: True
GPU name: NVIDIA GeForce RTX 3090
Torch device: cuda
